In [48]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

In [80]:
s1=0.53  # scaling during each step
a=1 # length of the loop's arm

s1

0.53

In [81]:
assert s1>0.5

In [82]:
def affine_transform(M,v):
    """Construct matrix for 2D affine transform from 2x2 matrix M and 2x1 vector v.
    The output is a 3x3 matrix M_af. The output of the affine transform is defined as:
    
    M*[[x1],[y1]] + v = [[x2],[y2]]
    
    Which is equivalent to (using M_af):
    
    M_af*[[x1],[y1],[1]]=[[x2],[y2],[1]]
    """
    result=np.zeros(shape=(3,3))
    # put the matrix M into top-left corner
    result[0:2,0:2]=M
    # put the vector v into top of the last column
    result[0:2,-1]=v
    # set last diagonal element to 1
    result[-1,-1]=1
    return np.matrix(result)

In [83]:
s2=(2*s1-1)*a  # loops on the corners of the "inverted loop" must be smaller
a2b=s1/(2*s2)  # ratio of the square loop to the arm, this value guarantees that an "inverted loop" has the same size as the original loop

S1=np.matrix(np.eye(2)*s1) # scaling matrices
S2=np.matrix(np.eye(2)*s2)
R=np.matrix([[0,-1],[1,0]]) # 90° counterclockwise rotation 

# length of the original loop size
b=a/a2b
# displacements for the smaller copies
d1=a*(1-s1)
d3=(s1+s2)*a
d2=d1

# transforms for each copy

# top right
M1=affine_transform(S1,[d1,0])
#bottom left
M2=affine_transform(S1,[0,-d1])
# top right small
M3=affine_transform(R**3*S2,[d3,-d2])
# bottom right small
M4=affine_transform(R**2*S2,[d3,-d3])
# bottom left small
M5=affine_transform(R*S2,[d2,-d3])


In [84]:
def affine_vector(vs):
    """Assumes that vs is an array filled with 2d vectors, i.e. either the last dimension is 2 or the lsat two dimensions are 2x1.
    Returns an array with the same shape, but each vector is turned into a 3x1 array by concatenating 1 to it."""
    extra_col=np.ones(shape=list(vs.shape)[:-1]+[1])
    result=np.append(vs,extra_col,-1)
    if result.shape[-1]>1:
        result=np.expand_dims(result,-1)
    return result

In [85]:
def make_step(lines):
    """Create all the transformed copies of the fractal by applying the affine transforms to an array of vectors."""
    lines_shape=lines.shape
    lines=np.reshape(lines,(-1,3,1))
    results=[np.array(np.matmul(M,lines)) for M in (M1,M2,M3,M4,M5)]
    results=[np.reshape(i,lines_shape) for i in results]
    return np.concatenate(results)        

In [89]:
def plot_affine(lines, path=None, title=None):
    """Plot the list of lines.
    Each line has the form 
    ```
    [[[x1],[y1],[1]],
    [[x2],[y2],[1]]]
    ```."""
    plt.figure(figsize=(10,10))
    plt.gca().set_aspect(1)
    for [[[x1],[y1],_],[[x2],[y2],_]] in lines: 
        plt.plot([x1,x2],[y1,y2],color="navy")
        
    if title:
        plt.title(title)
    if not path:
        plt.show()
    else:
        plt.savefig(path)

In [92]:
# initial lines
lines=np.array([[[a,0],[-b,0]],
      [[-b,0],[-b,b]],
      [[-b,b],[0,b]],
      [[0,b],[0,-a]]])

lines=affine_vector(lines)

for i in trange(7):
    plot_affine(lines,f"imgs/plot_{i}.png", title=f"step= {i}")
    plt.clf()
    lines=make_step(lines)


100%|██████████| 7/7 [00:45<00:00,  6.53s/it]


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [93]:
import imageio
from pathlib import Path
images = []
for filename in sorted(Path("imgs").glob("*.png")):
    images.append(imageio.imread(filename))
imageio.mimsave('movie.gif', images, duration=1)